# Preprocess Eng

In [ ]:
# import library
import re
import string
import progressbar
import numpy as np
import pandas as pd
from tqdm import tqdm 
import ipython_genutils
from sklearn import utils
from bs4 import BeautifulSoup
from keras.layers import Flatten
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.tokenize import WordPunctTokenizer
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer
from gensim.models.doc2vec import TaggedDocument

### load file 

In [ ]:
#load file
# file 1
dfct = pd.read_csv('data/clean_tweet.csv',index_col=0)
dfct['target']= dfct['target'].map({0:0,4:2})
dfct.dropna(inplace=True)
dfct.reset_index(drop=True,inplace=True)
dfct.info()
print()

#file 2
airlinedf = pd.read_csv('data/airline.csv',encoding='ISO-8859-1')
airlinedf['airline_sentiment'] = airlinedf['airline_sentiment'].map({'negative':0,'neutral':1,'positive':2})
airlinedf.dropna()
airlinedf.reset_index(drop=True,inplace=True)
airlinedf.info()
print()

#file 3
debatedf = pd.read_csv('data/debate.csv',encoding='ISO-8859-1')
debatedf['sentiment']=debatedf['sentiment'].map({'Negative':0,'Neutral':1,'Positive':2})
debatedf.info()

In [ ]:
# gabung data dari file
textdf=dfct['text'].astype('str')
textair=airlinedf['text'].astype('str')
textdebate=debatedf['text'].astype('str')
x=pd.concat([textdf,textair,textdebate])
x.reset_index(drop=True,inplace=True)

targetdf=dfct['target']
targetair=airlinedf['airline_sentiment']
targetdebate=debatedf['sentiment']
y=pd.concat([targetdf,targetair,targetdebate])
y.reset_index(drop=True,inplace=True)

df=pd.DataFrame({'text':x,'target':y})
df.reset_index(drop=True,inplace=True)
df.info()

### hapus emoji

In [ ]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  
                           u"\U0001F300-\U0001F5FF"  
                           u"\U0001F680-\U0001F6FF"  
                           u"\U0001F1E0-\U0001F1FF"  
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [ ]:
widgets = [progressbar.Percentage(),progressbar.Bar()," Processed : ",progressbar.Counter(),"  ",progressbar.ETA()]
bar = progressbar.ProgressBar(widgets=widgets, max_value=len(df.index))
bar.start()

t=[]
for index,row in bar(df.iterrows()):
    t.append(remove_emoji(row['text']))
df.text=t

### cleaning

In [ ]:
# tweet cleaning
tok=WordPunctTokenizer()
t=[]
for index,row in df.iterrows():
    x=row["text"]
    # hapus rt
    cl = re.sub(r'\s*RT\s*@[^:]*:.*', '', x)
    cl = re.sub(r'\s*rt\s*@[^:]*:.*', '', cl)
    # hapus mention
    cl = re.sub(r'@[A-Za-z0-9]([^:\s]+)+', '', cl)
    # hapus link
    cl = re.sub(r'https?://[A-Za-z0-9./]+', '', cl)
    # hapus hashtag
    cl = re.sub(r'(?:\s|^)#[A-Za-z0-9\-\.\_]+(?:\s|$)', '', cl)
    # kata ulang
    cl = re.sub(r'\w*\d\w*', '', cl)
    cl = re.sub(r'\b(\w+)(\1\b)+', r'\1', cl)
    # hapus simbol
    cl = re.sub(r'[^a-zA-Z]', ' ', cl)
    # lower
    cl=cl.lower()
    # format teks 
    cl=tok.tokenize(cl)
    cl=(" ".join(cl))
    t.append(cl)
    #print(cl)
df.text=t
df.head()

### hapus row yg terdapat data kosong

In [ ]:
# hapus data dgn missing value
ex=''
df=df[df.text != ex]
df.reset_index(drop=True,inplace=True)
df.info()

### hapus stopwords

In [ ]:
# stopwords
stopword_ = set(stopwords.words('english'))
t=[]
c=0
for index,row in df.iterrows():
    tokens = word_tokenize(row['text'])
    result = [i for i in tokens if not i in stopword_]
    result=' '.join(result)
    t.append(result)
    if c%100000==0:
        print('data processed : ',c)
    c+=1
df.text=t
df.head()

### stemming

In [ ]:
# stemming
stemmer= PorterStemmer()
t=[]
for index,row in df.iterrows():
    result=[]
    text = word_tokenize(row['text'])
    for word in text:
        result.append(stemmer.stem(word))
    result=' '.join(result) 
    t.append(result)
df.text=t

### save file

In [ ]:
# save ke csv
df.to_csv('clean_text.csv')